In [ ]:
from pathlib import Path

import h5py
import pandas as pd
import numpy as np
import functools
import matplotlib.pyplot as plt
import sklearn.feature_extraction.text
import torch

In [ ]:
PATH = Path('../input/shopee-product-matching')
if not PATH.is_dir():
    PATH = Path('/home/slex/data/shopee')

In [ ]:
TAKE_PAIRS = 5

In [ ]:
data = pd.read_csv(PATH/'test.csv')


In [ ]:
# def f1score(row, col):
#     val = row[col]
#     if type(val)==str:val = eval(val)
#     n = len( np.intersect1d(row.target,val) )
#     return 2*n / (len(row.target)+len(val))

# fake_test_df = pd.read_csv(PATH/'train.csv')[['posting_id', 'image', 'image_phash', 'title', 'label_group']]
# tmp = fake_test_df.groupby('label_group').posting_id.agg('unique').to_dict()
# fake_test_df['target'] = fake_test_df.label_group.map(tmp)

# data = fake_test_df

In [ ]:
sk_model = sklearn.feature_extraction.text.TfidfVectorizer(stop_words='english', binary=True, max_features=25_000)

In [ ]:
%%time
text_embeddings =sk_model.fit_transform(data.title)

In [ ]:
def csr_matrix_to_tensor(csr):
    coo = csr.tocoo()
    t = torch.sparse_coo_tensor([coo.row, coo.col], coo.data, csr.shape).cuda()
    return t

In [ ]:
sparse_embs = csr_matrix_to_tensor(text_embeddings)

In [ ]:
%%time
step = 5000
distances=[]
indices=[]
for chunk_start in range(0, text_embeddings.shape[0], step):
    chunk = text_embeddings[chunk_start:chunk_start+step]
    chunk = csr_matrix_to_tensor(chunk).to_dense()
    sim = sparse_embs @ chunk.T
    top_vals, top_inds = sim.topk(50, dim=0)
    distances.append(top_vals.T), indices.append(top_inds.T)

In [ ]:
distances = torch.cat(distances)
indices = torch.cat(indices)

In [ ]:
triplets=[]
for x in range(len(distances)):
    for y, val in zip(indices[x].tolist(), distances[x].tolist()):
        #if val < 1.2: triplets.append((x,y,val))
        triplets.append((x,y,val))

In [ ]:
triplets =sorted(triplets, key=lambda x: -x[2])

In [ ]:
triplets=triplets[ :int(len(data) * TAKE_PAIRS)]
len(triplets)

In [ ]:
groups = [[] for _ in range(len(data))]
for x,y,v in triplets:
    groups[x].append(y)

In [ ]:
if 'label_group' in data.columns.to_list():
    matches = [data.iloc[g].posting_id.to_list() for g in groups]
    data['matches'] = matches
    data['score']=data.apply(functools.partial(f1score, col='matches'),axis=1)
    print(data.score.mean())

In [ ]:
matches = [' '.join(data.iloc[g].posting_id.to_list()) for g in groups]
data['matches'] = matches

In [ ]:
data[['posting_id','matches']].to_csv('submission.csv',index=False)

In [ ]:
pd.read_csv('submission.csv')